## Задание 3.4

Даны результаты работы двух машинных переводчиков на небольших выборках переводов для разных языковых пар.

Стандартная оценка качества перевода производится с использованием специальной метрики BLEU.

Требуется определить:

* превосходит ли один переводчик в среднем по парам второй переводчик по переводу
* связано ли качество перевода для разных языковых пар для двух переводчиков?
При подсчете BLEU учитывать только слова, регистр не учитывать.

**Формат данных:**

Названиие файлов имеет формат lang1_lang2_<translator_id>.txt:

lang_1, lang_2 - языки (перевод с lang_1 на lang_2).

gold - эталонный вариант, с которым сравнивается перевод от систем машинного перевода.

In [20]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from nltk.translate import bleu_score
from nltk.tokenize import word_tokenize
from pprint import pprint

In [11]:
path = 'data/mt/'

langs = ['de_en', 'en_kk', 'en_ru', 'ru_de', 'ru_kk']
transls = ['_1.txt', '_2.txt', '_gold.txt']

corpus = {pair:{} for pair in langs}

In [12]:
corpus

{'de_en': {}, 'en_kk': {}, 'en_ru': {}, 'ru_de': {}, 'ru_kk': {}}

Считаем в словарь corpus данные наших переводов. Также сведем все слова к нижнему регистру и отфильтруем пустые строки в наших переводах.

In [31]:
for pair in langs:
    for trans_type in transls:
        new_path = path + pair + trans_type 
        with open(new_path, 'r', encoding='utf-8') as f:
            corpus[pair][trans_type] = f.read().lower().split('\n')
            corpus[pair][trans_type] = list(filter(lambda x: x if len(x) > 0 else None, corpus[pair][trans_type]))

Посмотрим, правильно ли все работает.

In [32]:
pprint(corpus['en_ru']['_gold.txt'][:10])

['через неделю , 1 сентября , германия начала вторжение в польшу .',
 '29 марта 2004 года литва вступила в нато .',
 'россия также подразделяется на 9 федеральных округов , в каждом из которых '
 'работает полномочный представитель президента россии .',
 'обладает правом издания указов , обязательных для исполнения на всей '
 'территории россии ( указы не должны противоречить федеральным законам ) .',
 'число занятых в промышленности — 27,5 % трудоспособного населения .',
 'традиционно в россии популярны настольные интеллектуальные игры .',
 '* сайт комиссии при президенте российской федерации по модернизации и '
 'технологическому развитию экономики россии .',
 'о статусе соционики существуют противоположные мнения .',
 '* лингвистика языка изучает язык как код , то есть систему объективно '
 'существующих социально закреплённых знаков и правил их употребления и '
 'сочетаемости .',
 'д .']


Реализуем функцию оценивания BLEU.

In [33]:
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method1

In [38]:
def BLEU(trans_1, trans_2):
    result = []
    
    for idx, sentence_1 in enumerate(trans_1):
        result.append(bleu_score.sentence_bleu(sentence_1, trans_2[idx], smoothing_function=smoothie))
    
    return np.cumsum(result)[-1]/len(trans_1)

In [43]:
for num, pair in enumerate(langs):
    print('For {} pair: BLEU_1 = {:.3f}, BLEU_2 = {:.3f}, difference = {}'.format(langs[num], BLEU(corpus[pair][transls[0]], corpus[pair][transls[2]]),
                                                                                    BLEU(corpus[pair][transls[1]], corpus[pair][transls[2]]),
                                                                                    bool(BLEU(corpus[pair][transls[0]], corpus[pair][transls[2]]) - BLEU(corpus[pair][transls[1]], corpus[pair][transls[2]]) > 0)))
    #print(BLEU(corpus[pair][transls[0]], corpus[pair][transls[2]]))

For de_en pair: BLEU_1 = 0.006, BLEU_2 = 0.006, difference = True
For en_kk pair: BLEU_1 = 0.012, BLEU_2 = 0.012, difference = True
For en_ru pair: BLEU_1 = 0.012, BLEU_2 = 0.015, difference = False
For ru_de pair: BLEU_1 = 0.005, BLEU_2 = 0.005, difference = False
For ru_kk pair: BLEU_1 = 0.009, BLEU_2 = 0.009, difference = False


Из полученных выше результатов видно, что все зависит от пары языков, между которыми происходит перевод. Далее, для удобства использования статистических критериев, загоним наши результаты в DataFrame.

In [47]:
data = pd.DataFrame(columns=['BLEU_1', 'BLEU_2'])

In [49]:
for pair in langs:
    row = []
    row.append(BLEU(corpus[pair][transls[0]], corpus[pair][transls[2]]))
    row.append(BLEU(corpus[pair][transls[1]], corpus[pair][transls[2]]))
    row.append(bool(BLEU(corpus[pair][transls[0]], corpus[pair][transls[2]]) - BLEU(corpus[pair][transls[1]], corpus[pair][transls[2]]) > 0))
    data.loc[pair] = row

In [50]:
data

,BLEU_1,BLEU_2,Best
de_en,0.006103,0.006008,True
en_kk,0.011882,0.011845,True
en_ru,0.012175,0.015287,False
ru_de,0.005207,0.005221,False
ru_kk,0.008805,0.009498,False


Проверим гипотезу о нормальном распределении данных на основе критерия Шапиро.

In [54]:
print('Критерий Шапиро, p_value = {:.3f}'.format(st.shapiro(data['BLEU_1'] - data['BLEU_2'])[1]))

Критерий Шапиро, p_value = 0.013


Видно, что p-value получилось <0.05, значит мы отвергаем гипотезу о нормальном распределении данных.

Сформулируем проверяемые гипотезы и воспользуемся для их проверки непараметрическими критериями знаков, Уилкоксона.

**H0:** P(BLEU_1 > BLEU_2) = 1/2

**H1:** P(BLEU_1 > BLEU_2) != 1/2

In [52]:
print('Критерий Шапиро'.format(st.shapiro(data['BLEU_1'] - data['BLEU_2'])[1]))

(0.714332640171051, 0.013444613665342331)

In [58]:
from statsmodels.stats.descriptivestats import sign_test
print(sign_test(data['BLEU_1'] - data['BLEU_2']))

(-0.5, 1.0)


In [60]:
st.wilcoxon(data['BLEU_1'] - data['BLEU_2'])

C:\Users\Anton Sotnikov\Anaconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


WilcoxonResult(statistic=5.0, pvalue=0.5001842570707944)

Видно, что мы не можем отвергнуть нулевую гипотезу, а значит не можем сказать, какой из переводчиков лучше. Качество зависит от языковой пары.